# Pennylane: I. Introduction to Quantum Computing
## I.14. Multi-Qubit Gate Challenge

In [1]:
# preparation
import numpy as np
import pennylane as qml

### Codercise I.14.1 - The Bell states

There are four Bell states. These states form the Bell basis together.

$$|\psi_+\rangle=\frac{1}{\sqrt{2}}(|00\rangle+|11\rangle)$$
$$|\psi_-\rangle=\frac{1}{\sqrt{2}}(|00\rangle-|11\rangle)$$
$$|\phi_+\rangle=\frac{1}{\sqrt{2}}(|01\rangle+|10\rangle)$$
$$|\phi_-\rangle=\frac{1}{\sqrt{2}}(|01\rangle-|10\rangle)$$

**Task**: Implement four Bell states.

**Solution**:
- For $|\psi_+\rangle$, we know that the Hadamard gate with a qubit can gives us a superposition, $|+\rangle$. That $|+\rangle$ is entangled to the remaining qubit $|0\rangle$ with the CNOT gate.
- For $|\psi_-\rangle$, the implementation is similar to the previous one except that the negative phase is introduced. To create the state $|-\rangle$, we can add the Pauli X operation in front of the qubit that goes into the Hadamard. Then, the CNOT operation applies on $|-\rangle$ and $|0\rangle$. 
- For $|\phi_+\rangle$, the superposition state $|+\rangle$ is entangled with $|1\rangle$ via the CNOT. The state $|1\rangle$ is prepared with the Pauli X gate.
- For $|\phi_-\rangle$, the implementation is similar to the previous one except for the negative phase. That negative phase is prepared by the Pauli Z operation of the state $|+\rangle$. Then, the CNOT operation applies between $-|+\rangle$ and $|1\rangle$.

In [2]:
dev = qml.device("default.qubit", wires=2)

# Starting from the state |00>, implement a PennyLane circuit
# to construct each of the Bell basis states.


@qml.qnode(dev)
def prepare_psi_plus():
    ##################
    # YOUR CODE HERE #
    ##################

    # PREPARE (1/sqrt(2)) (|00> + |11>)
    qml.Hadamard(0)
    qml.CNOT(wires=[0, 1])
    return qml.state()


@qml.qnode(dev)
def prepare_psi_minus():
    ##################
    # YOUR CODE HERE #
    ##################

    # PREPARE (1/sqrt(2)) (|00> - |11>)
    qml.PauliX(0)
    qml.Hadamard(0)
    qml.CNOT(wires=[0, 1])
    return qml.state()


@qml.qnode(dev)
def prepare_phi_plus():
    ##################
    # YOUR CODE HERE #
    ##################

    # PREPARE  (1/sqrt(2)) (|01> + |10>)
    qml.Hadamard(0)
    qml.PauliX(1)
    qml.CNOT(wires=[0, 1])
    return qml.state()


@qml.qnode(dev)
def prepare_phi_minus():
    ##################
    # YOUR CODE HERE #
    ##################

    # PREPARE  (1/sqrt(2)) (|01> - |10>)
    qml.Hadamard(0)
    qml.PauliX(1)
    qml.PauliZ(0)    
    qml.CNOT(wires=[0, 1])
    return qml.state()


psi_plus = prepare_psi_plus()
psi_minus = prepare_psi_minus()
phi_plus = prepare_phi_plus()
phi_minus = prepare_phi_minus()

# Uncomment to print results
print(f"|ψ_+> = {psi_plus}")
print(f"|ψ_-> = {psi_minus}")
print(f"|ϕ_+> = {phi_plus}")
print(f"|ϕ_-> = {phi_minus}")

|ψ_+> = [0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
|ψ_-> = [ 0.70710678+0.j  0.        +0.j  0.        +0.j -0.70710678+0.j]
|ϕ_+> = [0.        +0.j 0.70710678+0.j 0.70710678+0.j 0.        +0.j]
|ϕ_-> = [ 0.        +0.j  0.70710678+0.j -0.70710678+0.j  0.        +0.j]


### Codercise I.14.2 - Quantum Multiplexer

In the **quantum multiplexer** operation, all $2^n$ possible cases of $n$ control qubits are implemented, and the target operation is a single-qubit rotation (a uniformly controlled rotation).

**Task**: Implement a 3-qubit circuit that has the following performance:
- If the first two qubits are both $|0\rangle$, do nothing
- If the first qubit is $|0\rangle$ and the second is $|1\rangle$, apply `PauliX()` to the third qubit
- If the first qubit is $|1\rangle$ and the second is $|0\rangle$, apply `PauliZ()` to the third qubit
- If the first two qubits are both $|1\rangle$, apply `PauliY()` to the third qubit

NOTE: No `if` statement!

**Solution**:
1. $|01\rangle$: To distinguish '01' from '11', we create two Pauli-X operations: one to modify state to distinguish; one to return to the unmodified state. Between those state preparations, there is the Toffoli gate. The idea is that,
    - if the first two qubits is '01', it is transformed into '11' temporarily to perform the required `PauliX()` operation.
    - else if the first two qubits is '11', it still needs to undergo a transformation. In a transformed state, it cannot perform the required operation.

    So, only if the first two qubits is '01', it can become '11' and hence, the operation. Any other states will not convert to '11' to make the operation.

2. $|10\rangle$: Once again, '10' is converted into '11' temporarily for the same reason. Any other states will not convert to '11' to make the operation. The Hadamard gate is applied to the target qubit to create a superposition. When this is combined with the Toffoli operation, the result is the Pauli-Z operation. After the `PauliZ()` equivalent operations, the states 'target superposition' and '11', are undone by adding another Hadamard and X gates.

3. $|11\rangle$: This time, we prepare only the target qubit. The target qubit is rotated around the Z-axis for $\frac{\pi}{2}$ radians as the S gate is applied. The purpose of the adjoint S gate to reverse the state preparation effect after the operation. The combined effect of the S and Toffoli gates is the required `PauliY()`.

In [3]:
dev = qml.device("default.qubit", wires=3)

# State of first 2 qubits
state = [0, 1]


@qml.qnode(device=dev)
def apply_control_sequence(state):
    # Set up initial state of the first two qubits
    if state[0] == 1:
        qml.PauliX(wires=0)
    if state[1] == 1:
        qml.PauliX(wires=1)

    # Set up initial state of the third qubit - use |->
    # so we can see the effect on the output
    qml.PauliX(wires=2)
    qml.Hadamard(wires=2)

    ##################
    # YOUR CODE HERE #
    ##################

    # IMPLEMENT THE MULTIPLEXER
    # IF STATE OF FIRST TWO QUBITS IS 01, APPLY X TO THIRD QUBIT
    qml.PauliX(0) # convert to 11 temporarily
    qml.Toffoli(wires=[0, 1, 2])
    qml.PauliX(0) # convert to 11 temporarily
    # IF STATE OF FIRST TWO QUBITS IS 10, APPLY Z TO THIRD QUBIT
    qml.PauliX(1) # convert to 11 temporarily
    qml.Hadamard(2)
    qml.Toffoli(wires=[0, 1, 2])
    qml.Hadamard(2)
    qml.PauliX(1) # convert to 11 temporarily
    # IF STATE OF FIRST TWO QUBITS IS 11, APPLY Y TO THIRD QUBIT
    qml.adjoint(qml.S)(2)
    qml.Toffoli(wires=[0, 1, 2])
    qml.S(2)
    return qml.state()


print(apply_control_sequence(state))

[ 0.        +0.j  0.        +0.j -0.70710678+0.j  0.70710678+0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]


This notebook is done by `Myanmar Youths` for `Womanium Quantum + AI 2024` program.
- <a href="https://www.linkedin.com/in/la-wun-nannda-b047681b5/"><u>La Wun Nannda</u></a>
- <a href="https://www.linkedin.com/in/chit-zin-win-46a2a3263/"><u>Chit Zin Win</u></a>